<h1><b>Automatic Differentiation and Gradients</h1></b>
<br>Automatic differentiation is useful for implementing machine learning algorithms such as backpropagation for traning neural networks.
<br>This notebook will show you ways to compute gradients with TensorFlow.

In [ ]:
import numpy as np
import matplotlib as plt
import tensorflow as tf